In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
import joblib

# Veriyi yükleme
df = pd.read_csv("../data/veri.csv")

# Eğitim ve test setlerini oluştur
X = df.drop("Günlük Fiyat", axis=1)
y = df["Günlük Fiyat"]


# Eğitim ve test setlerine ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [2]:
# Algoritma listesi
algorithms = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(),
    "Lasso Regression": Lasso(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest": RandomForestRegressor(),
    "Gradient Boosting": GradientBoostingRegressor(),
    "AdaBoost": AdaBoostRegressor(),
    "Support Vector Machine": SVR(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Neural Network (MLP)": MLPRegressor(max_iter=500)
}

# Modelleri eğit ve değerlendir
best_model = RandomForestRegressor(random_state=42)

best_rmse = float("inf")
results = []

for name, model in algorithms.items():
    print(f"Model: {name}")
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    rmse = mean_squared_error(y_test, predictions, squared=False)
    results.append((name, rmse))
    print(f"RMSE: {rmse:.4f}\n")
    if rmse < best_rmse:
        best_rmse = rmse
        best_model = model

# En iyi modeli kaydet
joblib.dump(best_model, "../models/eniyi.joblib")
print(f"En iyi model kaydedildi: {type(best_model).__name__} ile RMSE: {best_rmse:.4f}")


Model: Linear Regression
RMSE: 0.0000

Model: Ridge Regression
RMSE: 0.1268

Model: Lasso Regression
RMSE: 4.7104

Model: Decision Tree
RMSE: 14.2021

Model: Random Forest


c:\Users\elifv\anaconda3\envs\ML\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\elifv\anaconda3\envs\ML\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\elifv\anaconda3\envs\ML\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\elifv\anaconda3\envs\ML\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the r

RMSE: 9.3185

Model: Gradient Boosting
RMSE: 6.7611

Model: AdaBoost


c:\Users\elifv\anaconda3\envs\ML\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


RMSE: 46.8823

Model: Support Vector Machine


c:\Users\elifv\anaconda3\envs\ML\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Users\elifv\anaconda3\envs\ML\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


RMSE: 129.9541

Model: K-Neighbors Regressor
RMSE: 42.2332

Model: Neural Network (MLP)
RMSE: 0.3163

En iyi model kaydedildi: LinearRegression ile RMSE: 0.0000


c:\Users\elifv\anaconda3\envs\ML\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\elifv\anaconda3\envs\ML\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [3]:
print("\nModel Performans Sonuçları:")
for name, rmse in results:
    print(f"{name}: RMSE = {rmse:.4f}")



Model Performans Sonuçları:
Linear Regression: RMSE = 0.0000
Ridge Regression: RMSE = 0.1268
Lasso Regression: RMSE = 4.7104
Decision Tree: RMSE = 14.2021
Random Forest: RMSE = 9.3185
Gradient Boosting: RMSE = 6.7611
AdaBoost: RMSE = 46.8823
Support Vector Machine: RMSE = 129.9541
K-Neighbors Regressor: RMSE = 42.2332
Neural Network (MLP): RMSE = 0.3163


In [4]:
# Eğitim ve test setlerini oluştur
import pandas as pd

# Assuming the CSV file is in the same directory as the notebook

df = pd.read_csv("../data/veri.csv")

X = df.drop("Günlük Fiyat", axis=1)
y = df["Günlük Fiyat"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modeli eğitme
best_model = RandomForestRegressor(random_state=42)
best_model.fit(X_train, y_train)

# Modeli kaydet

from joblib import dump

dump(best_model, "../models/eniyi.joblib")
print("Model yeniden eğitildi ve kaydedildi.")


Model yeniden eğitildi ve kaydedildi.
